In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model1 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map="auto")
tokenizer1 = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [2]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 910
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 195
    })
    test: Dataset({
        features: ['text'],
        num_rows: 195
    })
})


In [5]:
import json
data = json.loads(dataset['test'][1]['text'])

In [21]:
print(data['dialog'][1]['text'])

Hai. My name is Tom and i really need your help.


In [64]:
print(len(data['dialog']))

29


In [52]:
messages = [{
    "role": "user",
    "content": data['dialog'][1]['text']
}]

In [ ]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
perplexity.compute(model_id="clm-gpu/checkpoint-138000",
                             add_start_token=False,
                             predictions=input_texts, 
                             local_file_only=True)

In [6]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024

nlls = []
prev_end_loc = 0
messages = []
i = 1
while i<29:
    for begin_loc in tqdm(range(0, seq_len, stride)):
        messages.append({"role": "user", "content":  data['dialog'][i]['text']})
        i+=2
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            
            outputs = model.generate(
                input_ids=input_ids,
                labels=target_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.5,
                top_k=50,
                top_p=0.95
        )
            val = (tokenizer.decode(outputs[0], skip_special_tokens=True)[0].split("[/INST]")[-1])
            outputs = model(input_ids, labels=target_ids)
            print(outputs.logits.shape)
            print(torch.log(torch.softmax(outputs.logits,dim=-1)))
            #decoded_output = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs.logits.argmax(-1)]
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            #print(outputs)
            neg_log_likelihood = outputs.loss
            print(neg_log_likelihood.shape)

        nlls.append(neg_log_likelihood)
        print(len(nlls))
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print(ppl)
    messages.append({"role": "assistant", "content": val})

  0%|          | 0/8 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
  0%|          | 0/8 [00:04<?, ?it/s]


torch.Size([1, 22, 32000])
tensor([[[-20.6981, -20.6551, -15.5066,  ..., -19.3895, -18.4305, -19.0555],
         [-18.3835, -17.8367, -13.7156,  ..., -14.1101, -15.0202, -13.6492],
         [-16.3907, -15.9679, -22.1857,  ..., -15.3744, -18.0782, -14.9067],
         ...,
         [-18.1543, -17.1757, -25.7851,  ..., -18.1718, -22.5146, -22.0420],
         [-17.8572, -15.6346, -22.2029,  ..., -19.1551, -23.3944, -23.0076],
         [-23.2699, -22.2592, -24.8754,  ..., -25.1683, -24.6605, -25.1176]]],
       device='cuda:3')
torch.Size([])
1
tensor(21.2249, device='cuda:3')


  0%|          | 0/8 [00:02<?, ?it/s]


torch.Size([1, 77, 32000])
tensor([[[-20.7354, -20.6925, -15.5313,  ..., -19.4229, -18.4678, -19.0909],
         [-18.3172, -17.7508, -13.7078,  ..., -14.0926, -15.0243, -13.6746],
         [-16.5105, -16.0730, -22.1863,  ..., -15.5403, -18.2195, -15.0703],
         ...,
         [-17.6761, -16.6546, -15.3109,  ..., -17.0941, -23.8577, -23.2601],
         [-18.5505, -16.3640, -10.5261,  ..., -19.9962, -22.3171, -21.1540],
         [-19.5804, -17.8523,  -6.7074,  ..., -22.5394, -19.7914, -20.2269]]],
       device='cuda:3')
torch.Size([])
2
tensor(21.4581, device='cuda:3')


...

torch.Size([1, 280, 32000])
tensor([[[-20.7459, -20.7029, -15.5442,  ..., -19.4373, -18.4803, -19.1033],
         [-18.3042, -17.7417, -13.7104,  ..., -14.0815, -15.0307, -13.6616],
         [-16.5492, -16.1234, -22.2318,  ..., -15.5150, -18.2298, -15.1518],
         ...,
         [-20.7989, -19.5099, -17.5157,  ..., -21.2843, -26.3556, -25.5528],
         [-16.9544, -14.9622, -10.9759,  ..., -19.7991, -20.0520, -19.3665],
         [-18.5403, -16.3812,  -0.1321,  ..., -19.6439, -20.1399, -19.4368]]],
       device='cuda:3')
torch.Size([])
12
tensor(17.5845, device='cuda:3')


  0%|          | 0/8 [00:00<?, ?it/s]


torch.Size([1, 299, 32000])
tensor([[[-20.7530, -20.7139, -15.5549,  ..., -19.4483, -18.4835, -19.1065],
         [-18.3657, -17.8188, -13.7700,  ..., -14.1235, -14.9888, -13.6431],
         [-16.3264, -15.9133, -22.2726,  ..., -15.3732, -18.0070, -14.8273],
         ...,
         [-21.6113, -20.3283, -14.8908,  ..., -21.2853, -26.2814, -25.7209],
         [-17.2927, -15.5227,  -8.8479,  ..., -18.6447, -18.7873, -18.4289],
         [-17.5598, -15.7029,  -0.3059,  ..., -18.2141, -18.6887, -18.4953]]],
       device='cuda:3')
torch.Size([])
13
tensor(17.4045, device='cuda:3')


  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([1, 310, 32000])
tensor([[[-20.7518, -20.7128, -15.5566,  ..., -19.4393, -18.4764, -19.0995],
         [-18.3700, -17.8309, -13.7997,  ..., -14.1473, -14.9833, -13.6649],
         [-16.3521, -15.9742, -22.4312,  ..., -15.3616, -17.9703, -14.8738],
         ...,
         [-19.7226, -18.1335, -12.8093,  ..., -19.4724, -23.8679, -23.0593],
         [-17.3994, -15.3698,  -7.3102,  ..., -18.9303, -19.5504, -18.8073],
         [-18.7913, -16.8792,  -0.0374,  ..., -19.3284, -19.9241, -19.2894]]],
       device='cuda:3')
torch.Size([])
14
tensor(17.2012, device='cuda:3')


In [95]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer1.pad_token = tokenizer1.eos_token
seq_len = 1024

nlls = []
prev_end_loc = 0
messages = []
i = 1
while i<29:
    for begin_loc in tqdm(range(0, seq_len, stride)):
        messages.append({"role": "user", "content":  data['dialog'][i]['text']})
        i+=2
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
        input_ids = tokenizer1.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            
            outputs = model1.generate(
                input_ids=input_ids,
                labels=target_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.5,
                top_k=50,
                top_p=0.95
        )
            val = (tokenizer1.decode(outputs[0], skip_special_tokens=True)[0].split("[/INST]")[-1])
            outputs = model1(input_ids, labels=target_ids)
            #decoded_output = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs.logits.argmax(-1)]
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            #print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            #print(outputs)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print(ppl)
    messages.append({"role": "assistant", "content": val})

  0%|          | 0/8 [00:02<?, ?it/s]


tensor(455.2411, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


tensor(333.4897, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


...

tensor(1179.7234, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]


tensor(1296.0780, device='cuda:1')


  0%|          | 0/8 [00:06<?, ?it/s]

tensor(1420.3734, device='cuda:1')


In [93]:
import torch
from tqdm import tqdm

#max_length = model.config.n_positions
stride = 512
text = "Absolutely, building this relationship means being there for each other through thick and thin. Whether it's sharing our joys, navigating challenges, or simply lending an ear, I'm dedicated to fostering a connection grounded in trust, empathy, and genuine care. Together, we can create a safe space where open dialogue and mutual support flourish."
encodings = tokenizer.encode_plus("Of course! I'm here to help and chat with you whenever you need.", return_tensors="pt").to('cuda')
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

100%|██████████| 1/1 [00:00<00:00,  7.56it/s]


In [71]:
print(outputs.logits.shape)

torch.Size([1, 18, 32000])


In [94]:

print(torch.mean(torch.max(torch.softmax(outputs.logits, dim=-1), dim=-1)[0]))

tensor(0.6309, device='cuda:1')


In [84]:

print(torch.max(torch.softmax(outputs.logits, dim=-1),dim=-1))

torch.return_types.max(
values=tensor([[0.4461, 0.4940, 0.7285, 0.4279, 0.1381, 0.9768, 0.9906, 0.7321, 0.3723]],
       device='cuda:1'),
indices=tensor([[22478,   264, 10360,   354,   272,   266, 28127,  7768,    13]],
       device='cuda:1'))


In [92]:

print(torch.mean(torch.max(torch.softmax(outputs.logits, dim=-1), dim=-1)[0]))

tensor(0.6380, device='cuda:1')


In [1]:
import numpy as np

data = np.load("/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/instruct-mistral/evaluate/perplexity_scores/llama/llama_base.npy", allow_pickle=True)

print(data)

[[143.4096     31.732403   16.633274   11.515069    9.100802    7.727912
    6.8666377   6.260498 ]
 [ 29.021667   15.979132   10.856503    8.518468    7.2484775   6.444537
    5.8987575   5.5092688]
 [109.79915    33.625156   18.313675   12.627146    9.882281    8.280714
    7.2584205   6.5607643]
 [ 29.021667   16.490791   11.03499     8.539032    7.284541    6.5211115
    6.0132794   5.627395 ]
 [ 86.880264   33.24064    18.463526   13.013455   10.339102    8.805614
    7.8480744   7.169255 ]]
